In [8]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph
from graph import DataExtractor

In [2]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [3]:
voltai_graph = DataExtractor(
    folder_path="./data/input_data/", 
    file_number=139
).graph

In [4]:
# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

In [5]:
# 4개의 질문 입력
questions = [
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "CAM (Cathode Active Material)": {
                "Stoichiometry information": "NCM-622",
                "Commercial NCM": "No",
                "Lithium source": "LiOH",
                "Synthesis method": "co-precipitation",
                "Crystallization method": "Hydrothermal",
                "Crystallization temperature": "100°C",
                "Crystallization time": "12 hr",
                "Doping": "Zr4+ doping",
                "Coating": "ZrO2 coating",
                "Additional treatment": "None"
                }
        }
    ]
    """,    
    
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "Electrode (only for coin-cell (half-cell))": {
                "Active material : Conductive additive : Binder ratio": "90 : 5 : 5",
                "Electrolyte": "LiPF6 (EC, EMC, DEC mixture in a 1:1:1 volume ratio)",
                "Additive": "FEC 10% addition",
                "Electrode thickness": "100 µm",
                "Only Cathode Electrode diameter": "14π",
                "Loading density (mass loading of NCM)": "0.005 g/cm^2",
                "Additional treatment for electrode": "None"
                },
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {        
            "Morphological results": {
                "Explanation of SEM results": "Fig. 2a, b; the NCM-622 seems to have more or less a spherical morphology with a diameter of 3–5 µm, composed of densely packed primary particles",
                "Explanation of TEM results": "None"
                },    
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {            
            "Cathode Performance": {
                "Capacity at all C-rate, mAh/g (with electrode state)": [{
                "214.5 mAh/g": "@0.1C, ZrO2-coated",
                "200.8 mAh/g": "@0.5C, ZrO2-coated"
                }],
                "Voltage range": "2.8–4.3 V",
                "Temperature": "Room temperature and 55°C"
                }
        }
    ]
    """
]


In [24]:
# 4개의 질문에 대해 그래프 실행 및 출력
outputs = []
for i, question in enumerate(questions):
    # inputs = GraphState(question=question)
    print(f"#####   {i}번째 질문   #####")
    # print(question)
    result = voltai_graph.invoke(
        input={"question":question},
        config=config,
    )
    outputs.append(result)

##   0번째 질문
==== [RELEVANCE CHECK] ====
yes
    0번째 질문 관련성 통과
##   1번째 질문
==== [RELEVANCE CHECK] ====
yes
    1번째 질문 관련성 통과
##   2번째 질문
==== [RELEVANCE CHECK] ====
yes
    2번째 질문 관련성 통과
##   3번째 질문
==== [RELEVANCE CHECK] ====
yes
    3번째 질문 관련성 통과


In [ ]:
import pprint
outputs_dict = outputs[0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[2]["answer"][0]["Morphological results"] | outputs[3]["answer"][0]["Cathode Performance"]
pprint.pprint(outputs_dict)

[{'answer': [{'CAM (Cathode Active Material)': {'Additional treatment': 'freeze '
                                                                        'drying '
                                                                        'or '
                                                                        'heat '
                                                                        'drying',
                                                'Coating': 'Al2O3 coating',
                                                'Commercial NCM': 'No',
                                                'Crystallization method': 'calcination',
                                                'Crystallization temperature': '750°C',
                                                'Crystallization time': '20 hr',
                                                'Doping': 'None',
                                                'Lithium source': 'LiOH',
                                                'S

In [28]:
# pd.DataFrame(outputs_dict)